<a href="https://colab.research.google.com/github/choisangh/STUDY/blob/main/DL/NLP/%EA%B0%90%EC%A0%95%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* ref : https://techblog-history-younghunjo1.tistory.com/111

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings(action='ignore')


#-------------------- 차트 관련 속성 (한글처리, 그리드) -----------
#plt.rc('font', family='NanumGothicOTF') # For MacOS
plt.rcParams['font.family']= 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
sns.set()

#-------------------- 주피터 , 출력결과 넓이 늘리기 ---------------
from IPython.core.display import display, HTML
display(HTML("<style>.container{width:100% !important;}</style>"))
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('max_colwidth', None)

#-------------------- 차트 관련 속성 (한글처리, 그리드) -----------
plt.rcParams['font.family']= 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
sns.set()

In [ ]:
import requests
import json
from tqdm import tqdm

In [ ]:
# colab
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
# 데이터 로드
from sklearn.datasets import fetch_20newsgroups
# subset인자에 all로 train, test데이터 모두 가져오기
news_data = fetch_20newsgroups(subset='all', random_state=42)
# 데이터 분할
from sklearn.datasets import fetch_20newsgroups
#remove인자에 있는 부분은 해당 기사의 일부분을 제거한 것이다. 왜냐하면 오로지 기사의 body(본문)에 해당하는 텍스트들로만 학습시키려 했기 때문
train_data = fetch_20newsgroups(subset='train',
                               remove=('headers','footers','quotes'),
                               random_state=12)
X_train = train_data.data
y_train = train_data.target

test_data = fetch_20newsgroups(subset='test',
                              remove=('headers','footers','quotes'),
                              random_state=12)
X_test = test_data.data
y_test = test_data.target
print(f"학습 데이터 크기: {len(X_train)}\n 테스트 데이터 크기: {len(X_test)}")

학습 데이터 크기: 11314
 테스트 데이터 크기: 7532


In [ ]:
# 텍스트 데이터 feature Vectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

cnt_vect = CountVectorizer() #텍스트 벡터화
# CountVecorizer도 한번 fit 하면 검증, 테스트 데이터에도 똑같은 것으로 해야함!
# 다시 한 번 fit하게 되면 벡터화되는 feature 개수가 달라짐!
X_train_vec = cnt_vect.fit_transform(X_train)
X_test_vec = cnt_vect.transform(X_test)

# 로지스틱 리그레션으로 모델링하기
lr_clf = LogisticRegression() #분류
lr_clf.fit(X_train_vec, y_train)
y_pred = lr_clf.predict(X_test_vec)
acc = accuracy_score(y_test, y_pred)
print(f"분류 정확도 : {acc : .4f}")

분류 정확도 :  0.6063


<img src = "https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fd7CWe5%2FbtqGG1kV1lD%2F9M6ujgYrB6DLCeJuYvuIW0%2Fimg.png"></img>

In [ ]:
# Tf-idf로 feature vectorizer 시킨 후 로지스틱리그레션 해보기
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

tf_idf = TfidfVectorizer()
X_train_tf = tf_idf.fit_transform(X_train)
X_test_tf = tf_idf.transform(X_test)

lr_clf = LogisticRegression()
lr_clf.fit(X_train_tf, y_train)
y_pred = lr_clf.predict(X_test_tf)
acc = accuracy_score(y_test, y_pred)
print(f"Tf-idf 벡터화 후 정확도 :{acc : .4f}")

Tf-idf 벡터화 후 정확도 : 0.6737


<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2F9FHO4%2FbtqGK6ejxWD%2FcmoLoWtdk2BndsykK0ZQ00%2Fimg.png"></img>

# 2. 리뷰 감성 분류하기

## 2-1. 지도학습에 기반한 감성 분석

In [ ]:
# Pipeline을 통해서 텍스트를 벡터화시키고 모델 학습시키기!
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, roc_auc_score
import re
import pandas as pd
import numpy as np
# tsv 파일도 csv호출함수로 불러올 수 있음!
# headers=0이면 헤더값(칼럼이름) 출력, 1이면 헤더값 없이 바로 출력
# quoting=3 이면 ""(큰따음포) 인용구는 무시하고 출력함!
review_df = pd.read_csv('labeledTrainData.tsv', encoding='utf-8',
                       header=0, sep='\t', quoting=3)
# 정규표현식으로 HTML태그와 숫자 삭제하기
# Series의 replace함수로 HTML태그 없애주기
review_df['review'] = review_df['review'].str.replace('<br />',' ')
# re.sub('패턴','뭘로바꿀지',바꿀문자열)
review_df['review'] = review_df['review'].apply(lambda x : re.sub('[^a-zA-Z]',' ',x))

# 학습, 테스트 데이터 분리
target = review_df['sentiment']
feature = review_df['review']

X_train, X_test, y_train, y_test = train_test_split(feature,
                                                   target,
                                                   test_size=0.3,
                                                   random_state=42)
# Vectorizer인자로는 불용어 제거, ngram범위 설정해주기
pipeline = Pipeline([('cnt_vect',CountVectorizer(stop_words='english',
                                               ngram_range=(1,2))),
                    ('lr_clf', LogisticRegression(C=10))])
# 위에서 설정해준 Pipleline으로 데이터 학습시킬 때는 벡터화시키기 전의 원본 데이터
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
# '1' label로 예측될 확률 추출 for roc_auc_score
y_pred_proba = pipeline.predict_proba(X_test)[:,1]
acc = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_proba)
print(f"정확도:{acc : .3f}\nAUC score:{auc : .3f}")         

FileNotFoundError: ignored

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbWCisd%2FbtqGG0GiRXb%2Fik6hfgIrnBjVe3H6ZHUK00%2Fimg.png">

## 2-2. 비지도 학습기반 감성분석
* 감성 사전 필요

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer  #nltk에서 제공하는 감성 사전 <영어만 가능>

senti_analyzer = SentimentIntensityAnalyzer()
# polarity_score로 하나의 텍스트에 대해 각 부정/객관/긍정 그리고 총 합한 감성지수 출력(-1~1)
senti_score = senti_analyzer.polarity_scores(review_df['review'][0])
print(senti_score)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score
from sklearn.metrics import recall_score, f1_score

# 임계치설정(보통 0.1)을 통해 compound(총 감성지수)가 임계치값보다 높으면 긍정, 낮으면 부정으로 분석
def get_sentiment(review, threshold):
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(review)
    
    compound_score = scores['compound']
    final_sentiment = 1 if compound_score >= threshold else 0
    return final_sentiment

 # 각 텍스트 데이터에 위에서 설정한 감성 label 얻는 함수 적용하기
# 임계값은 0.1로 설정
review_df['vader_pred'] = review_df['review'].apply(lambda x : get_sentiment(x, 0.1))
# 원본 데이터에서 주어진 정답 label과 VADER로 예측한 label 비교
y_target = review_df['sentiment']
y_pred = review_df['vader_pred']

print(confusion_matrix(y_target, y_pred))
print("정확도 :", accuracy_score(y_target, y_pred))
print("정밀도 :", precision_score(y_target, y_pred))
print("재현율 :", recall_score(y_target, y_pred))
print("F1 score :", f1_score(y_target, y_pred))